In [1]:
import torch
from transformers import AutoModel, AutoTokenizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import numpy as np


ModuleNotFoundError: No module named 'torch'

Bước 1: Chuẩn Bị Dữ Liệu

In [ ]:
queries = ["PhoBERT là một mô hình mạnh mẽ.", "Học sâu là tương lai của AI."]
documents = ["PhoBERT được phát triển bởi VinAI.", "Học sâu cần nhiều dữ liệu và tính toán."]
labels = [1, 0, 2, 0]  # Cần đảm bảo số lượng nhãn khớp với số mẫu


Bước 2: Trích Xuất Đặc Trưng từ PhoBERT

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")
model = AutoModel.from_pretrained("vinai/phobert-base")

def get_embeddings(texts):
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state[:, 0, :].cpu().numpy()  # Sử dụng embedding của token [CLS]

query_embeddings = get_embeddings(queries)
document_embeddings = get_embeddings(documents)

# In kích thước của các embeddings
print(f"Query embeddings shape: {query_embeddings.shape}")  # (2, 768)
print(f"Document embeddings shape: {document_embeddings.shape}")  # (2, 768)

Bước 3: Kết Hợp Embeddings và Kiểm Tra Dữ Liệu

In [ ]:
# Kết hợp embeddings của queries và documents
X = np.concatenate((query_embeddings, document_embeddings), axis=0)
y = np.array(labels)

# In kích thước và nội dung của X và y
print(f"Shape of X: {X.shape}")  # (4, 768)
print(f"Shape of y: {y.shape}")  # (2,)
print("X:", X)
print("y:", y)

# Đảm bảo rằng y có đúng dạng (số mẫu,)
assert len(y.shape) == 1, "y phải có dạng (số mẫu,)"
assert X.shape[0] == y.shape[0], "Số lượng mẫu trong X và y phải khớp nhau"


Bước 4: Huấn Luyện Mô Hình SVM

In [ ]:
svm_model = SVC(kernel='linear')
svm_model.fit(X, y)

B5: Đánh giá mô hình

In [ ]:
# Tạo dữ liệu kiểm tra mẫu
test_queries = ["PhoBERT là một công cụ hữu ích.", "AI cần sự tham gia của con người."]
test_documents = ["PhoBERT giúp phân tích văn bản tốt hơn.", "AI có thể tự động hóa nhiều công việc."]

test_query_embeddings = get_embeddings(test_queries)
test_document_embeddings = get_embeddings(test_documents)

# In kích thước của các embeddings kiểm tra
print(f"Test query embeddings shape: {test_query_embeddings.shape}")  # (2, 768)
print(f"Test document embeddings shape: {test_document_embeddings.shape}")  # (2, 768)

# Kết hợp embeddings của queries và documents cho tập kiểm tra
X_test = np.concatenate((test_query_embeddings, test_document_embeddings), axis=0)
y_test = np.array([1, 0, 1, 0])  # Nhãn thực tế cho dữ liệu kiểm tra

# In kích thước và nội dung của X_test và y_test
print(f"Shape of X_test: {X_test.shape}")  # (4, 768)
print(f"Shape of y_test: {y_test.shape}")  # (4,)
print("X_test:", X_test)
print("y_test:", y_test)

# Dự đoán với mô hình SVM
y_pred = svm_model.predict(X_test)

# In kích thước và nội dung của y_pred
print(f"Shape of y_pred: {y_pred.shape}")  # (4,)
print("y_pred:", y_pred)

# Tính toán độ chính xác
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")